# Router Quality Classifier Training (Part Three)

Cost as a prompt property could be the only thing that may matter in many cases. Majority of the LLMs can generate a good response for a lot of promots, this can be see as many of the benchmarks are becoming obsolete. Users don't always have complex queries for LLMs to answer. Going from GPT-4o to GPT-5 many users are not seeing much improvement, other than tech/AI influencers' praises for the model. Mainly because only a small percentage of queries might need responses from a strong model. This is where cost can be optimised. In this part, we train to predict if the query would need a strong model to query or not.


From our (extremely) small dataset we see that the trends holds true, `meta-llama/llama-3.1-8b-instruct` is capable of answering majority (~66%) of the queries. But this is under the assumption that the model `openai/gpt-oss-120b` is a good judge, this can be challenged and researched further.

This is what the trend in our data looks like:

```
=== RATING DISTRIBUTION ===
Rating 1: 37 (2.1%)
Rating 2: 107 (6.1%)
Rating 3: 86 (4.9%)
Rating 4: 329 (18.7%)
Rating 5: 1197 (68.2%)
========================================
```

Everything below rating 5 can be considered as something that should be addressed by a strong model to keep quality high.  

Regarding model selection for this task, `openai/gpt-oss-120b` stands way ahead of our weak model i.e. `meta-llama/llama-3.1-8b-instruct` in benchmarks but the weak model costs substantially less. Hence for the sake of setting an example, these models align to the framework set by the paper RouteLLM.

We evaluate the F1 score of the optimised models here instead of the accuracy as there is a clear imbalance in the dataset.

Selection of model to train, since we have barely any datapoints to actually make learning happen we opt for a model with not many parameters which also gives us speed as bigger models can be resouce heavy. In a perfect world with hundreds of thousands of data points (like in the `routellm/gpt4_dataset` dataset and better with availability of better GPUs, we'd opt for a newer and larger embedding model to finetune or follow what the routeLLM github demonstrates i.e. finetuning a Llama 3.1 8B LLM on Anyscale's infrastructure.

The tl;dr outcome of this notebook is that this method is inappropriate for a tiny dataset, hence the overfit and low f1 score.

All the plots can recreated on your W&B account, make sure you are loggedin when you run this notebook.

*Note:* A response generated that is of good quality should be routed to a weak model (label class 0) and a response that is of bad quality should be routed to a strong model (label class 1). We may be refering to our classes as good quality/weak model and bad quality/strong model, interchangebly in this notebook.

## 1. Import Dependencies

In [2]:
import time
import torch
import os
import json
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, set_seed
from sklearn.model_selection import train_test_split
import random
import numpy as np
import wandb
from tqdm import tqdm
from torch.optim.lr_scheduler import LinearLR, ReduceLROnPlateau
from sklearn.metrics import f1_score
import pandas as pd
from typing import Dict, Any, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

print("Dependencies imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")

Dependencies imported successfully!
PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3060


## 2. Utility Functions and Classes

In [3]:
def set_reproducible_environment(seed: int = 9999) -> None:
    """Set up reproducible environment with fixed random seeds."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
    # Set transformers seed
    set_seed(seed)
    
    print(f"Reproducible environment set with seed: {seed}")

In [4]:
def load_rated_dataset(file_path: str) -> list:
    """Load the rated dataset from JSONL file."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                item = json.loads(line.strip())
                if 'rating' in item and item['rating'] is not None:
                    data.append(item)
            except json.JSONDecodeError:
                continue
    return data

def analyze_dataset(df: pd.DataFrame, rating_threshold: int = 4) -> None:
    """Analyze and print dataset statistics."""
    total_samples = len(df)
    labels_array = df["quality_label"].values
    label_0_count = np.sum(labels_array == 0)
    label_1_count = np.sum(labels_array == 1)
    
    print("\n=== DATASET ANALYSIS ===")
    print(f"Total samples: {total_samples}")
    print(f"Label 0 (Good quality, rating >= {rating_threshold}): {label_0_count} ({label_0_count/total_samples*100:.1f}%)")
    print(f"Label 1 (Poor quality, rating < {rating_threshold}): {label_1_count} ({label_1_count/total_samples*100:.1f}%)")
    if label_1_count > 0:
        print(f"Class balance ratio (0:1): {label_0_count/label_1_count:.2f}:1")
    
    print("\n=== RATING DISTRIBUTION ===")
    rating_counts = df['rating'].value_counts().sort_index()
    for rating, count in rating_counts.items():
        print(f"Rating {rating}: {count} ({count/total_samples*100:.1f}%)")
    print("=" * 40)

In [5]:
class RatedDataset(Dataset):
    """Custom PyTorch dataset for rated response data."""
    
    def __init__(self, sentences: list, labels: list):
        self.sentences = sentences
        self.labels = labels

    def __len__(self) -> int:
        return len(self.sentences)

    def __getitem__(self, idx: int) -> Tuple[str, torch.Tensor]:
        sentence = self.sentences[idx]
        label = self.labels[idx]
        return sentence, torch.tensor(label, dtype=torch.float)

def worker_init_fn(worker_id: int, seed: int = 9999) -> None:
    """Ensure deterministic behavior in DataLoader workers."""
    np.random.seed(seed + worker_id)
    random.seed(seed + worker_id)

In [6]:
class CompactQualityClassifier(nn.Module):
    """Compact transformer-based quality classifier optimized for limited data."""
    
    def __init__(self, transformer_model_name: str, dropout_rate: float = 0.2, 
                 freeze_transformer: bool = False):
        super(CompactQualityClassifier, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(transformer_model_name)
        self.transformer = AutoModel.from_pretrained(transformer_model_name)
        
        # Freeze transformer parameters if specified
        if freeze_transformer:
            for param in self.transformer.parameters():
                param.requires_grad = False
            print("Transformer layers frozen - only training classification head")
        else:
            print("All transformer layers will be fine-tuned")
        
        # Add dropout for regularization
        self.dropout = nn.Dropout(dropout_rate)
        
        # Simplified architecture for smaller dataset
        hidden_size = self.transformer.config.hidden_size
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 1)
        )

    def forward(self, sentences: list) -> torch.Tensor:
        # Tokenize and encode as batch
        encoding = self.tokenizer(
            sentences, 
            padding=True, 
            truncation=True, 
            max_length=512,
            return_tensors="pt"
        ).to(next(self.transformer.parameters()).device)
        
        outputs = self.transformer(**encoding)
        # Use the [CLS] token embedding for classification
        pooled = outputs.last_hidden_state[:, 0, :]
        
        # Apply dropout and classification layers
        x = self.dropout(pooled)
        logits = self.classifier(x)
        
        return logits
    
    def count_parameters(self) -> Tuple[int, int]:
        """Count total and trainable parameters."""
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        
        print(f"\n=== MODEL PARAMETER ANALYSIS ===")
        print(f"Total parameters: {total_params:,}")
        print(f"Trainable parameters: {trainable_params:,}")
        print(f"Frozen parameters: {total_params - trainable_params:,}")
        print(f"Percentage trainable: {trainable_params/total_params*100:.1f}%")
        print("=" * 40)
        
        return total_params, trainable_params

## 3. Training and Validation Functions

In [7]:
def get_metric_for_early_stopping(metric_name: str, loss: float, accuracy: float, f1: float) -> float:
    """Get the metric value based on the early stopping metric name."""
    if metric_name == "loss":
        return loss
    elif metric_name == "accuracy":
        return accuracy
    elif metric_name == "f1":
        return f1
    else:
        raise ValueError(f"Unknown metric: {metric_name}. Choose from 'loss', 'accuracy', 'f1'")

def is_metric_improved(metric_name: str, current_value: float, best_value: float) -> bool:
    """Check if the metric has improved (lower is better for loss, higher is better for accuracy/f1)."""
    if metric_name == "loss":
        return current_value < best_value
    else:  # accuracy or f1
        return current_value > best_value

def validate(model: nn.Module, val_loader: DataLoader, criterion: nn.Module, 
            device: torch.device) -> Tuple[float, float, float]:
    """Perform validation and return the loss, accuracy, and F1 score."""
    model.eval()
    valid_loss = 0.0
    valid_correct = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for sentences, labels in val_loader:
            sentences = list(sentences)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(sentences).squeeze(1)
            
            # Compute loss
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            
            # Calculate accuracy and collect predictions for F1
            predictions = torch.round(torch.sigmoid(outputs))
            valid_correct += (predictions == labels).sum().item()
            
            # Store for F1 calculation
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    valid_loss /= len(val_loader)
    valid_accuracy = valid_correct / len(val_loader.dataset)
    
    # Handle case where all predictions are the same class
    try:
        valid_f1 = f1_score(all_labels, all_predictions, average='binary')
    except:
        valid_f1 = 0.0
    
    return valid_loss, valid_accuracy, valid_f1

## 4. Main Training Function

In [8]:
def train_router_quality_classifier(
    dataset_path: str = "rated_enhanced_dataset_meta_llama_llama_3.1_8b_instruct_20250811_140130_20250811_151904.jsonl",
    # Model hyperparameters
    transformer_model_name: str = 'distilbert-base-uncased',
    dropout_rate: float = 0.1,
    freeze_transformer: bool = False,
    
    # Training hyperparameters
    learning_rate: float = 0.0001,
    batch_size: int = 16,
    n_epochs: int = 10,
    weight_decay: float = 0.0001,
    gradient_clip_value: float = 1.0,
    
    # Data hyperparameters
    test_size: float = 0.2,
    dataset_fraction: float = 1.0,
    rating_threshold: int = 4,
    
    # Early stopping and scheduling
    early_stopping_metric: str = "f1",
    early_stopping_patience: int = 500,
    warmup_steps: int = 50,
    enable_lr_annealing: bool = True,
    lr_annealing_patience: int = 5,
    lr_annealing_factor: float = 0.5,
    lr_annealing_min_lr: float = 1e-6,
    
    # Experiment settings
    experiment_name: str = "router_training",
    seed: int = 9999,
    use_wandb: bool = True
) -> Dict[str, Any]:
    """
    Train a router quality classifier with comprehensive configuration options.
    
    Returns:
        Dictionary containing training results and best metrics
    """
    
    # Set up reproducible environment
    set_reproducible_environment(seed)
    
    # Generate run name
    timestamp = time.strftime('%Y%m%d_%H%M%S')
    run_name = f"{experiment_name}_{timestamp}" if experiment_name else f"run_{timestamp}"
    
    # Initialize W&B if enabled
    if use_wandb:
        wandb.init(project="router_rated", name=run_name)
    
    print(f"\nStarting training run: {run_name}")
    print(f"Using dataset: {dataset_path}")
    
    # Load and process dataset
    print("\nLoading rated dataset...")
    data_items = load_rated_dataset(dataset_path)
    print(f"Loaded {len(data_items)} items with ratings")
    
    # Convert to DataFrame and filter
    df = pd.DataFrame(data_items)
    df = df.dropna(subset=['rating'])
    print(f"Dataset after filtering: {len(df)} items")
    
    # Create binary labels
    df["quality_label"] = df["rating"].apply(lambda x: 0 if x >= rating_threshold else 1)
    
    # Analyze dataset
    analyze_dataset(df, rating_threshold)
    
    # Extract sentences and labels
    sentences = df["model_response"].tolist()
    labels = df["quality_label"].tolist()
    
    # Sample dataset if needed
    if dataset_fraction < 1.0:
        df_sampled = df.sample(frac=dataset_fraction, random_state=seed).reset_index(drop=True)
        sentences = df_sampled["model_response"].tolist()
        labels = df_sampled["quality_label"].tolist()
        print(f"Sampled {len(sentences)} items ({dataset_fraction*100}% of dataset)")
    
    # Split data
    sentences_train, sentences_val, labels_train, labels_val = train_test_split(
        sentences, labels, test_size=test_size, random_state=seed, stratify=labels
    )
    
    print(f"\nTraining samples: {len(sentences_train)}")
    print(f"Validation samples: {len(sentences_val)}")
    
    # Create datasets and data loaders
    train_data = RatedDataset(sentences_train, labels_train)
    val_data = RatedDataset(sentences_val, labels_val)
    
    # Create generator for reproducible shuffling
    g = torch.Generator()
    g.manual_seed(seed)
    
    train_loader = DataLoader(
        train_data, 
        batch_size=batch_size, 
        shuffle=True,
        generator=g,
        worker_init_fn=lambda w: worker_init_fn(w, seed),
        num_workers=0
    )
    
    val_loader = DataLoader(
        val_data, 
        batch_size=batch_size, 
        shuffle=False,
        worker_init_fn=lambda w: worker_init_fn(w, seed),
        num_workers=0
    )
    
    print("DataLoaders created successfully")
    
    # Initialize model
    print(f"\nInitializing model: {transformer_model_name}")
    model = CompactQualityClassifier(
        transformer_model_name=transformer_model_name,
        dropout_rate=dropout_rate,
        freeze_transformer=freeze_transformer
    )
    
    # Deterministic weight initialization for classifier layers
    def init_weights_deterministic(module):
        if isinstance(module, nn.Linear):
            torch.manual_seed(seed)
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
    
    model.classifier.apply(init_weights_deterministic)
    
    # Move to device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    print(f"📱 Model moved to device: {device}")
    
    # Count parameters
    total_params, trainable_params = model.count_parameters()
    
    # Initialize training components
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    # Learning rate scheduler for warmup
    scheduler = LinearLR(optimizer, start_factor=0.1, total_iters=warmup_steps)
    
    # Learning rate annealing scheduler
    lr_annealing_scheduler = None
    if enable_lr_annealing:
        lr_annealing_scheduler = ReduceLROnPlateau(
            optimizer, 
            mode='min',
            factor=lr_annealing_factor,
            patience=lr_annealing_patience,
            min_lr=lr_annealing_min_lr,
            verbose=True
        )
        print(f"LR Annealing enabled: factor={lr_annealing_factor}, patience={lr_annealing_patience}")
    
    # Create runs directory
    runs_dir = "runs"
    os.makedirs(runs_dir, exist_ok=True)
    
    # Initialize tracking variables
    best_valid_loss = float('inf')
    best_valid_accuracy = 0.0
    best_valid_f1 = 0.0
    best_metric_value = float('inf') if early_stopping_metric == "loss" else 0.0
    patience_counter = 0
    
    # Log hyperparameters to W&B
    if use_wandb:
        wandb.config = {
            "learning_rate": learning_rate,
            "epochs": n_epochs,
            "batch_size": batch_size,
            "transformer_model": transformer_model_name,
            "dataset_fraction": dataset_fraction,
            "test_size": test_size,
            "experiment_name": experiment_name,
            "weight_decay": weight_decay,
            "warmup_steps": warmup_steps,
            "early_stopping_patience": early_stopping_patience,
            "early_stopping_metric": early_stopping_metric,
            "enable_lr_annealing": enable_lr_annealing,
            "dropout_rate": dropout_rate,
            "freeze_transformer": freeze_transformer,
            "rating_threshold": rating_threshold,
            "total_parameters": total_params,
            "trainable_parameters": trainable_params,
            "dataset_size": len(sentences),
            "seed": seed,
            "device": str(device),
        }
    
    # Initial validation
    print("\nStarting initial validation...")
    initial_valid_loss, initial_valid_accuracy, initial_valid_f1 = validate(model, val_loader, criterion, device)
    print(f"Initial - Loss: {initial_valid_loss:.4f}, Accuracy: {initial_valid_accuracy:.4f}, F1: {initial_valid_f1:.4f}")
    
    if use_wandb:
        wandb.log({
            "epoch": 0,
            "valid_loss": initial_valid_loss,
            "valid_accuracy": initial_valid_accuracy,
            "valid_f1": initial_valid_f1,
        })
    
    # Training loop
    print(f"\nStarting training for {n_epochs} epochs...")
    global_step = 0
    
    def set_all_seeds(s):
        """Set all random seeds for maximum reproducibility."""
        random.seed(s)
        np.random.seed(s)
        torch.manual_seed(s)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(s)
    
    for epoch in range(n_epochs):
        # Re-set seeds for each epoch
        set_all_seeds(seed + epoch)
        
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_predictions = []
        train_labels_list = []
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}")
        
        for batch_idx, (sentences, labels) in enumerate(progress_bar):
            sentences = list(sentences)
            labels = labels.to(device)
            
            # Clear gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(sentences).squeeze(1)
            
            # Compute loss
            loss = criterion(outputs, labels)
            
            # Backward pass with gradient clipping
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_value)
            optimizer.step()
            
            # Update learning rate scheduler for warmup period
            if global_step < warmup_steps:
                scheduler.step()
            
            global_step += 1
            
            # Track training metrics
            train_loss += loss.item()
            predictions = torch.round(torch.sigmoid(outputs))
            train_correct += (predictions == labels).sum().item()
            
            # Store for F1 calculation
            train_predictions.extend(predictions.detach().cpu().numpy())
            train_labels_list.extend(labels.detach().cpu().numpy())
            
            # Update progress bar
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'lr': f'{optimizer.param_groups[0]["lr"]:.6f}'
            })
        
        # Calculate epoch metrics
        train_loss /= len(train_loader)
        train_accuracy = train_correct / len(train_loader.dataset)
        
        try:
            train_f1 = f1_score(train_labels_list, train_predictions, average='binary')
        except:
            train_f1 = 0.0
        
        # Validation
        valid_loss, valid_accuracy, valid_f1 = validate(model, val_loader, criterion, device)
        
        # Apply LR annealing
        if enable_lr_annealing and lr_annealing_scheduler is not None:
            lr_annealing_scheduler.step(train_loss)
        
        # Log metrics
        if use_wandb:
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": train_loss,
                "train_accuracy": train_accuracy,
                "train_f1": train_f1,
                "valid_loss": valid_loss,
                "valid_accuracy": valid_accuracy,
                "valid_f1": valid_f1,
                "learning_rate": optimizer.param_groups[0]["lr"],
            })
        
        print(f'Epoch {epoch+1}/{n_epochs}:')
        print(f'  Training   - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, F1: {train_f1:.4f}')
        print(f'  Validation - Loss: {valid_loss:.4f}, Accuracy: {valid_accuracy:.4f}, F1: {valid_f1:.4f}')
        print(f'  Learning Rate: {optimizer.param_groups[0]["lr"]:.8f}')
        
        # Early stopping and model saving
        current_metric_value = get_metric_for_early_stopping(early_stopping_metric, valid_loss, valid_accuracy, valid_f1)
        
        if is_metric_improved(early_stopping_metric, current_metric_value, best_metric_value):
            best_metric_value = current_metric_value
            best_valid_loss = valid_loss
            best_valid_accuracy = valid_accuracy
            best_valid_f1 = valid_f1
            patience_counter = 0
            torch.save(model.state_dict(), os.path.join(runs_dir, f'best_model_{run_name}.pt'))
            print(f"New best model saved with {early_stopping_metric}: {current_metric_value:.4f}")
            
            if use_wandb:
                wandb.log({
                    "best_valid_loss": best_valid_loss,
                    "best_valid_accuracy": best_valid_accuracy,
                    "best_valid_f1": best_valid_f1,
                    f"best_{early_stopping_metric}": best_metric_value,
                })
        else:
            patience_counter += 1
            print(f"No improvement in {early_stopping_metric}. Patience: {patience_counter}/{early_stopping_patience}")
            
            if patience_counter >= early_stopping_patience:
                print(f'Early stopping triggered after {epoch+1} epochs (no improvement in {early_stopping_metric})')
                break
    
    # Final evaluation
    print("\nFinal model evaluation...")
    model.load_state_dict(torch.load(os.path.join(runs_dir, f'best_model_{run_name}.pt')))
    final_valid_loss, final_valid_accuracy, final_valid_f1 = validate(model, val_loader, criterion, device)
    
    print(f'\nTRAINING COMPLETE')
    print(f'Best validation loss: {best_valid_loss:.4f}')
    print(f'Best validation accuracy: {best_valid_accuracy:.4f}')
    print(f'Best validation F1: {best_valid_f1:.4f}')
    print(f'Best {early_stopping_metric}: {best_metric_value:.4f}')
    
    # Log final results
    if use_wandb:
        wandb.log({
            "final_valid_loss": final_valid_loss,
            "final_valid_accuracy": final_valid_accuracy,
            "final_valid_f1": final_valid_f1,
            "training_completed": True,
            "epochs_trained": epoch + 1,
            "early_stopped": patience_counter >= early_stopping_patience,
        })
        
        # Create summary table
        summary_table = wandb.Table(
            columns=["Metric", "Best Validation", "Final Validation"],
            data=[
                ["Loss", best_valid_loss, final_valid_loss],
                ["Accuracy", best_valid_accuracy, final_valid_accuracy],
                ["F1", best_valid_f1, final_valid_f1]
            ]
        )
        wandb.log({"validation_summary": summary_table})
        wandb.finish()
    
    # Return results
    return {
        "run_name": run_name,
        "model_path": os.path.join(runs_dir, f'best_model_{run_name}.pt'),
        "best_metrics": {
            "loss": best_valid_loss,
            "accuracy": best_valid_accuracy,
            "f1": best_valid_f1,
            early_stopping_metric: best_metric_value
        },
        "final_metrics": {
            "loss": final_valid_loss,
            "accuracy": final_valid_accuracy,
            "f1": final_valid_f1
        },
        "training_info": {
            "epochs_trained": epoch + 1,
            "early_stopped": patience_counter >= early_stopping_patience,
            "total_parameters": total_params,
            "trainable_parameters": trainable_params
        }
    }

## 5. Training

### Basic Training with Default Parameters

In [8]:
# Basic training run with default parameters
results = train_router_quality_classifier(
    experiment_name="basic_test",
    n_epochs=3,  # Quick test
    use_wandb=False  # Disable wandb for testing
)

print("\nTraining Results:")
print(f"Model saved at: {results['model_path']}")
print(f"Best F1 score: {results['best_metrics']['f1']:.4f}")
print(f"Total epochs: {results['training_info']['epochs_trained']}")

Reproducible environment set with seed: 9999

Starting training run: basic_test_20250813_005702
Using dataset: rated_enhanced_dataset_meta_llama_llama_3.1_8b_instruct_20250811_140130_20250811_151904.jsonl

Loading rated dataset...
Loaded 1756 items with ratings
Dataset after filtering: 1756 items

=== DATASET ANALYSIS ===
Total samples: 1756
Label 0 (Good quality, rating >= 4): 1526 (86.9%)
Label 1 (Poor quality, rating < 4): 230 (13.1%)
Class balance ratio (0:1): 6.63:1

=== RATING DISTRIBUTION ===
Rating 1: 37 (2.1%)
Rating 2: 107 (6.1%)
Rating 3: 86 (4.9%)
Rating 4: 329 (18.7%)
Rating 5: 1197 (68.2%)

Training samples: 1404
Validation samples: 352
DataLoaders created successfully

Initializing model: distilbert-base-uncased
All transformer layers will be fine-tuned
All transformer layers will be fine-tuned
📱 Model moved to device: cuda

=== MODEL PARAMETER ANALYSIS ===
Total parameters: 66,576,257
Trainable parameters: 66,576,257
Frozen parameters: 0
Percentage trainable: 100.0%
LR 

Epoch 1/3: 100%|██████████| 88/88 [00:50<00:00,  1.75it/s, loss=0.3028, lr=0.000100]



Epoch 1/3:
  Training   - Loss: 0.4040, Accuracy: 0.8583, F1: 0.0569
  Validation - Loss: 0.3556, Accuracy: 0.8693, F1: 0.0000
  Learning Rate: 0.00010000
No improvement in f1. Patience: 1/500


Epoch 2/3: 100%|██████████| 88/88 [00:50<00:00,  1.73it/s, loss=0.0724, lr=0.000100]



Epoch 2/3:
  Training   - Loss: 0.3602, Accuracy: 0.8796, F1: 0.2283
  Validation - Loss: 0.3520, Accuracy: 0.8750, F1: 0.1200
  Learning Rate: 0.00010000
New best model saved with f1: 0.1200
New best model saved with f1: 0.1200


Epoch 3/3: 100%|██████████| 88/88 [00:51<00:00,  1.72it/s, loss=0.6044, lr=0.000100]



Epoch 3/3:
  Training   - Loss: 0.3335, Accuracy: 0.8974, F1: 0.4375
  Validation - Loss: 0.3310, Accuracy: 0.8864, F1: 0.3103
  Learning Rate: 0.00010000
New best model saved with f1: 0.3103

Final model evaluation...
New best model saved with f1: 0.3103

Final model evaluation...

TRAINING COMPLETE
Best validation loss: 0.3310
Best validation accuracy: 0.8864
Best validation F1: 0.3103
Best f1: 0.3103

Training Results:
Model saved at: runs\best_model_basic_test_20250813_005702.pt
Best F1 score: 0.3103
Total epochs: 3

TRAINING COMPLETE
Best validation loss: 0.3310
Best validation accuracy: 0.8864
Best validation F1: 0.3103
Best f1: 0.3103

Training Results:
Model saved at: runs\best_model_basic_test_20250813_005702.pt
Best F1 score: 0.3103
Total epochs: 3


### Lets Adjust Parameters

In [9]:
intial_results = train_router_quality_classifier(
    # Model configuration
    dropout_rate=0.3,
    
    # Training hyperparameters
    learning_rate=0.0001,
    n_epochs=30,
    weight_decay=0.01,
    
    # Learning rate scheduling
    enable_lr_annealing=True,
    
    # Experiment settings
    experiment_name="dropout30wd01",
    use_wandb=True  # Enable wandb logging
)

Reproducible environment set with seed: 9999


wandb: Currently logged in as: shubhamjune (shubhamjune-vilnius-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Starting training run: dropout30wd01_20250813_161421
Using dataset: rated_enhanced_dataset_meta_llama_llama_3.1_8b_instruct_20250811_140130_20250811_151904.jsonl

Loading rated dataset...
Loaded 1756 items with ratings
Dataset after filtering: 1756 items

=== DATASET ANALYSIS ===
Total samples: 1756
Label 0 (Good quality, rating >= 4): 1526 (86.9%)
Label 1 (Poor quality, rating < 4): 230 (13.1%)
Class balance ratio (0:1): 6.63:1

=== RATING DISTRIBUTION ===
Rating 1: 37 (2.1%)
Rating 2: 107 (6.1%)
Rating 3: 86 (4.9%)
Rating 4: 329 (18.7%)
Rating 5: 1197 (68.2%)

Training samples: 1404
Validation samples: 352
DataLoaders created successfully

Initializing model: distilbert-base-uncased
All transformer layers will be fine-tuned
📱 Model moved to device: cuda

=== MODEL PARAMETER ANALYSIS ===
Total parameters: 66,576,257
Trainable parameters: 66,576,257
Frozen parameters: 0
Percentage trainable: 100.0%
LR Annealing enabled: factor=0.5, patience=5

Starting initial validation...
Initial - 

Epoch 1/30:  16%|█████████████████████▏                                                                                                               | 14/88 [00:08<00:43,  1.69it/s, loss=0.4647, lr=0.000035]


KeyboardInterrupt: 

## We have a tiny dataset, therefore we need high regularisation to stop from overfitting.

In [10]:
do70 = train_router_quality_classifier(
    # Model configuration
    dropout_rate=0.3,
    
    # Training hyperparameters
    learning_rate=0.0001,
    n_epochs=30,
    weight_decay=0.01,
    
    # Learning rate scheduling
    enable_lr_annealing=True,
    
    # Experiment settings
    experiment_name="dropout70wd01",
    use_wandb=True  # Enable wandb logging
)

Reproducible environment set with seed: 9999


epoch,▁
valid_accuracy,▁
valid_f1,▁
valid_loss,▁
epoch,0
valid_accuracy,0.2358
valid_f1,0.20178
valid_loss,0.73829



Starting training run: dropout70wd01_20250813_161447
Using dataset: rated_enhanced_dataset_meta_llama_llama_3.1_8b_instruct_20250811_140130_20250811_151904.jsonl

Loading rated dataset...
Loaded 1756 items with ratings
Dataset after filtering: 1756 items

=== DATASET ANALYSIS ===
Total samples: 1756
Label 0 (Good quality, rating >= 4): 1526 (86.9%)
Label 1 (Poor quality, rating < 4): 230 (13.1%)
Class balance ratio (0:1): 6.63:1

=== RATING DISTRIBUTION ===
Rating 1: 37 (2.1%)
Rating 2: 107 (6.1%)
Rating 3: 86 (4.9%)
Rating 4: 329 (18.7%)
Rating 5: 1197 (68.2%)

Training samples: 1404
Validation samples: 352
DataLoaders created successfully

Initializing model: distilbert-base-uncased
All transformer layers will be fine-tuned
📱 Model moved to device: cuda

=== MODEL PARAMETER ANALYSIS ===
Total parameters: 66,576,257
Trainable parameters: 66,576,257
Frozen parameters: 0
Percentage trainable: 100.0%
LR Annealing enabled: factor=0.5, patience=5

Starting initial validation...
Initial - 

Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.86it/s, loss=0.3741, lr=0.000100]


Epoch 1/30:
  Training   - Loss: 0.4266, Accuracy: 0.8476, F1: 0.0446
  Validation - Loss: 0.3660, Accuracy: 0.8693, F1: 0.0000
  Learning Rate: 0.00010000
No improvement in f1. Patience: 1/500


...

Epoch 29/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:48<00:00,  1.82it/s, loss=0.0005, lr=0.000100]


Epoch 29/30:
  Training   - Loss: 0.0001, Accuracy: 1.0000, F1: 1.0000
  Validation - Loss: 1.5068, Accuracy: 0.8807, F1: 0.3824
  Learning Rate: 0.00010000
No improvement in f1. Patience: 15/500


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:48<00:00,  1.83it/s, loss=0.0000, lr=0.000100]


Epoch 30/30:
  Training   - Loss: 0.0001, Accuracy: 1.0000, F1: 1.0000
  Validation - Loss: 1.5573, Accuracy: 0.8807, F1: 0.3824
  Learning Rate: 0.00010000
No improvement in f1. Patience: 16/500

Final model evaluation...

TRAINING COMPLETE
Best validation loss: 0.8819
Best validation accuracy: 0.8807
Best validation F1: 0.4000
Best f1: 0.4000


best_f1,▁▅▆▇█
best_valid_accuracy,▇▃▁██
best_valid_f1,▁▅▆▇█
best_valid_loss,▁▃▇██
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
epochs_trained,▁
final_valid_accuracy,▁
final_valid_f1,▁
final_valid_loss,▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▂▃▄▅▆▇▇▇█████████████████████


In [11]:
do50wdlow = train_router_quality_classifier(
    # Model configuration
    dropout_rate=0.3,
    
    # Training hyperparameters
    learning_rate=0.0001,
    n_epochs=30,
    weight_decay=0.0001,
    
    # Learning rate scheduling
    enable_lr_annealing=True,
    
    # Experiment settings
    experiment_name="do50wdlow",
    use_wandb=True  # Enable wandb logging
)

Reproducible environment set with seed: 9999



Starting training run: do50wdlow_20250813_164111
Using dataset: rated_enhanced_dataset_meta_llama_llama_3.1_8b_instruct_20250811_140130_20250811_151904.jsonl

Loading rated dataset...
Loaded 1756 items with ratings
Dataset after filtering: 1756 items

=== DATASET ANALYSIS ===
Total samples: 1756
Label 0 (Good quality, rating >= 4): 1526 (86.9%)
Label 1 (Poor quality, rating < 4): 230 (13.1%)
Class balance ratio (0:1): 6.63:1

=== RATING DISTRIBUTION ===
Rating 1: 37 (2.1%)
Rating 2: 107 (6.1%)
Rating 3: 86 (4.9%)
Rating 4: 329 (18.7%)
Rating 5: 1197 (68.2%)

Training samples: 1404
Validation samples: 352
DataLoaders created successfully

Initializing model: distilbert-base-uncased
All transformer layers will be fine-tuned
📱 Model moved to device: cuda

=== MODEL PARAMETER ANALYSIS ===
Total parameters: 66,576,257
Trainable parameters: 66,576,257
Frozen parameters: 0
Percentage trainable: 100.0%
LR Annealing enabled: factor=0.5, patience=5

Starting initial validation...
Initial - Loss

Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:48<00:00,  1.82it/s, loss=0.2890, lr=0.000100]


Epoch 1/30:
  Training   - Loss: 0.4226, Accuracy: 0.8462, F1: 0.0442
  Validation - Loss: 0.3588, Accuracy: 0.8693, F1: 0.0000
  Learning Rate: 0.00010000
No improvement in f1. Patience: 1/500


Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:48<00:00,  1.83it/s, loss=0.1378, lr=0.000100]


...

Epoch 29/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.0000, lr=0.000100]


Epoch 29/30:
  Training   - Loss: 0.0000, Accuracy: 1.0000, F1: 1.0000
  Validation - Loss: 1.9221, Accuracy: 0.8750, F1: 0.3889
  Learning Rate: 0.00010000
No improvement in f1. Patience: 19/500


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.0000, lr=0.000100]


Epoch 30/30:
  Training   - Loss: 0.0000, Accuracy: 1.0000, F1: 1.0000
  Validation - Loss: 1.9324, Accuracy: 0.8750, F1: 0.3889
  Learning Rate: 0.00010000
No improvement in f1. Patience: 20/500

Final model evaluation...

TRAINING COMPLETE
Best validation loss: 1.3900
Best validation accuracy: 0.8778
Best validation F1: 0.3944
Best f1: 0.3944


best_f1,▁▃▅▇██
best_valid_accuracy,▇█▇▅▁▇
best_valid_f1,▁▃▅▇██
best_valid_loss,▁▁▂▂▃█
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
epochs_trained,▁
final_valid_accuracy,▁
final_valid_f1,▁
final_valid_loss,▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▂▃▅▆▇▇███████████████████████


In [12]:
slow_lr = train_router_quality_classifier(
    # Model configuration
    dropout_rate=0.3,
    
    # Training hyperparameters
    learning_rate=0.00005,
    n_epochs=30,
    weight_decay=0.0001,
    
    # Learning rate scheduling
    enable_lr_annealing=True,
    
    # Experiment settings
    experiment_name="slow_lr",
    use_wandb=True  # Enable wandb logging
)

Reproducible environment set with seed: 9999



Starting training run: slow_lr_20250813_170739
Using dataset: rated_enhanced_dataset_meta_llama_llama_3.1_8b_instruct_20250811_140130_20250811_151904.jsonl

Loading rated dataset...
Loaded 1756 items with ratings
Dataset after filtering: 1756 items

=== DATASET ANALYSIS ===
Total samples: 1756
Label 0 (Good quality, rating >= 4): 1526 (86.9%)
Label 1 (Poor quality, rating < 4): 230 (13.1%)
Class balance ratio (0:1): 6.63:1

=== RATING DISTRIBUTION ===
Rating 1: 37 (2.1%)
Rating 2: 107 (6.1%)
Rating 3: 86 (4.9%)
Rating 4: 329 (18.7%)
Rating 5: 1197 (68.2%)

Training samples: 1404
Validation samples: 352
DataLoaders created successfully

Initializing model: distilbert-base-uncased
All transformer layers will be fine-tuned
📱 Model moved to device: cuda

=== MODEL PARAMETER ANALYSIS ===
Total parameters: 66,576,257
Trainable parameters: 66,576,257
Frozen parameters: 0
Percentage trainable: 100.0%
LR Annealing enabled: factor=0.5, patience=5

Starting initial validation...
Initial - Loss: 

Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.3044, lr=0.000050]


Epoch 1/30:
  Training   - Loss: 0.4266, Accuracy: 0.8355, F1: 0.0494
  Validation - Loss: 0.3692, Accuracy: 0.8693, F1: 0.0000
  Learning Rate: 0.00005000
No improvement in f1. Patience: 1/500


Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.0939, lr=0.000050]


...

Epoch 29/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:48<00:00,  1.81it/s, loss=0.0006, lr=0.000025]


Epoch 29/30:
  Training   - Loss: 0.0001, Accuracy: 1.0000, F1: 1.0000
  Validation - Loss: 1.3615, Accuracy: 0.8778, F1: 0.2951
  Learning Rate: 0.00002500
No improvement in f1. Patience: 15/500


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:48<00:00,  1.82it/s, loss=0.0000, lr=0.000025]


Epoch 30/30:
  Training   - Loss: 0.0001, Accuracy: 1.0000, F1: 1.0000
  Validation - Loss: 1.4002, Accuracy: 0.8778, F1: 0.2951
  Learning Rate: 0.00002500
No improvement in f1. Patience: 16/500

Final model evaluation...

TRAINING COMPLETE
Best validation loss: 1.4006
Best validation accuracy: 0.8665
Best validation F1: 0.4471
Best f1: 0.4471


best_f1,▁▃▄▇█
best_valid_accuracy,█▇▆▁▁
best_valid_f1,▁▃▄▇█
best_valid_loss,▁▁▂▃█
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
epochs_trained,▁
final_valid_accuracy,▁
final_valid_f1,▁
final_valid_loss,▁
learning_rate,████████████████████████▁▁▁▁▁▁
train_accuracy,▁▃▃▅▆▇▇███████████████████████


In [13]:
very_slow_lr = train_router_quality_classifier(
    # Model configuration
    dropout_rate=0.3,
    
    # Training hyperparameters
    learning_rate=0.000007,
    n_epochs=30,
    weight_decay=0.0001,
    
    # Learning rate scheduling
    enable_lr_annealing=False,
    
    # Experiment settings
    experiment_name="very_slow_lr",
    use_wandb=True  # Enable wandb logging
)

Reproducible environment set with seed: 9999



Starting training run: very_slow_lr_20250813_180851
Using dataset: rated_enhanced_dataset_meta_llama_llama_3.1_8b_instruct_20250811_140130_20250811_151904.jsonl

Loading rated dataset...
Loaded 1756 items with ratings
Dataset after filtering: 1756 items

=== DATASET ANALYSIS ===
Total samples: 1756
Label 0 (Good quality, rating >= 4): 1526 (86.9%)
Label 1 (Poor quality, rating < 4): 230 (13.1%)
Class balance ratio (0:1): 6.63:1

=== RATING DISTRIBUTION ===
Rating 1: 37 (2.1%)
Rating 2: 107 (6.1%)
Rating 3: 86 (4.9%)
Rating 4: 329 (18.7%)
Rating 5: 1197 (68.2%)

Training samples: 1404
Validation samples: 352
DataLoaders created successfully

Initializing model: distilbert-base-uncased
All transformer layers will be fine-tuned
📱 Model moved to device: cuda

=== MODEL PARAMETER ANALYSIS ===
Total parameters: 66,576,257
Trainable parameters: 66,576,257
Frozen parameters: 0
Percentage trainable: 100.0%

Starting initial validation...
Initial - Loss: 0.7383, Accuracy: 0.2358, F1: 0.2018

St

Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.85it/s, loss=0.3129, lr=0.000007]


Epoch 1/30:
  Training   - Loss: 0.5000, Accuracy: 0.7678, F1: 0.1641
  Validation - Loss: 0.3890, Accuracy: 0.8693, F1: 0.0000
  Learning Rate: 0.00000700
No improvement in f1. Patience: 1/500


Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.2049, lr=0.000007]


...

Epoch 29/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.0001, lr=0.000007]


Epoch 29/30:
  Training   - Loss: 0.0004, Accuracy: 1.0000, F1: 1.0000
  Validation - Loss: 1.5155, Accuracy: 0.8778, F1: 0.2712
  Learning Rate: 0.00000700
No improvement in f1. Patience: 14/500


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.0000, lr=0.000007]


Epoch 30/30:
  Training   - Loss: 0.0032, Accuracy: 0.9986, F1: 0.9946
  Validation - Loss: 1.4932, Accuracy: 0.8750, F1: 0.3125
  Learning Rate: 0.00000700
No improvement in f1. Patience: 15/500

Final model evaluation...

TRAINING COMPLETE
Best validation loss: 1.0569
Best validation accuracy: 0.8722
Best validation F1: 0.3478
Best f1: 0.3478


best_f1,▁▆▇██
best_valid_accuracy,▂█▅█▁
best_valid_f1,▁▆▇██
best_valid_loss,▁▁▂▂█
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
epochs_trained,▁
final_valid_accuracy,▁
final_valid_f1,▁
final_valid_loss,▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▄▄▄▄▄▅▅▆▇▇███████████████████


In [14]:
very_slow_lr_withA = train_router_quality_classifier(
    # Model configuration
    dropout_rate=0.3,
    
    # Training hyperparameters
    learning_rate=0.000007,
    n_epochs=30,
    weight_decay=0.0001,
    
    # Learning rate scheduling
    enable_lr_annealing=True,
    
    # Experiment settings
    experiment_name="very_slow_lr_withA",
    use_wandb=True  # Enable wandb logging
)

Reproducible environment set with seed: 9999



Starting training run: very_slow_lr_withA_20250813_183459
Using dataset: rated_enhanced_dataset_meta_llama_llama_3.1_8b_instruct_20250811_140130_20250811_151904.jsonl

Loading rated dataset...
Loaded 1756 items with ratings
Dataset after filtering: 1756 items

=== DATASET ANALYSIS ===
Total samples: 1756
Label 0 (Good quality, rating >= 4): 1526 (86.9%)
Label 1 (Poor quality, rating < 4): 230 (13.1%)
Class balance ratio (0:1): 6.63:1

=== RATING DISTRIBUTION ===
Rating 1: 37 (2.1%)
Rating 2: 107 (6.1%)
Rating 3: 86 (4.9%)
Rating 4: 329 (18.7%)
Rating 5: 1197 (68.2%)

Training samples: 1404
Validation samples: 352
DataLoaders created successfully

Initializing model: distilbert-base-uncased
All transformer layers will be fine-tuned
📱 Model moved to device: cuda

=== MODEL PARAMETER ANALYSIS ===
Total parameters: 66,576,257
Trainable parameters: 66,576,257
Frozen parameters: 0
Percentage trainable: 100.0%
LR Annealing enabled: factor=0.5, patience=5

Starting initial validation...
Initi

Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.3130, lr=0.000007]


Epoch 1/30:
  Training   - Loss: 0.5000, Accuracy: 0.7678, F1: 0.1641
  Validation - Loss: 0.3890, Accuracy: 0.8693, F1: 0.0000
  Learning Rate: 0.00000700
No improvement in f1. Patience: 1/500


Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.2053, lr=0.000007]


...

Epoch 29/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.0001, lr=0.000007]


Epoch 29/30:
  Training   - Loss: 0.0007, Accuracy: 1.0000, F1: 1.0000
  Validation - Loss: 1.4321, Accuracy: 0.8722, F1: 0.3284
  Learning Rate: 0.00000700
No improvement in f1. Patience: 21/500


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.0000, lr=0.000007]


Epoch 30/30:
  Training   - Loss: 0.0057, Accuracy: 0.9993, F1: 0.9973
  Validation - Loss: 1.4559, Accuracy: 0.8807, F1: 0.3438
  Learning Rate: 0.00000700
No improvement in f1. Patience: 22/500

Final model evaluation...

TRAINING COMPLETE
Best validation loss: 0.4045
Best validation accuracy: 0.8864
Best validation F1: 0.3548
Best f1: 0.3548


best_f1,▁▆▆█
best_valid_accuracy,▁█▇█
best_valid_f1,▁▆▆█
best_valid_loss,▄▁▇█
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
epochs_trained,▁
final_valid_accuracy,▁
final_valid_f1,▁
final_valid_loss,▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▄▄▄▄▄▅▅▆▆▇███████████████████


### Hyperparameter Comparison

In [10]:
# Compare different configurations
configs = [
    {
        "name": "low_dropout",
        "dropout_rate": 0.1,
        "learning_rate": 0.0001,
        "n_epochs": 30
    },
    {
        "name": "high_dropout", 
        "dropout_rate": 0.5,
        "learning_rate": 0.0001,
        "n_epochs": 30
    },
    {
        "name": "higher_lr",
        "dropout_rate": 0.3,
        "learning_rate": 0.0005,
        "n_epochs": 30
    }
]

comparison_results = []

for config in configs:
    print(f"\nTesting configuration: {config['name']}")
    
    result = train_router_quality_classifier(
        experiment_name=f"comparison_{config['name']}",
        dropout_rate=config['dropout_rate'],
        learning_rate=config['learning_rate'],
        n_epochs=30,
        use_wandb=True
    )
    
    comparison_results.append({
        "config": config['name'],
        "f1_score": result['best_metrics']['f1'],
        "accuracy": result['best_metrics']['accuracy'],
        "epochs": result['training_info']['epochs_trained']
    })

# Display comparison
print("\nConfiguration Comparison:")
print("-" * 60)
print(f"{'Config':<15} {'F1 Score':<10} {'Accuracy':<10} {'Epochs':<8}")
print("-" * 60)
for result in comparison_results:
    print(f"{result['config']:<15} {result['f1_score']:<10.4f} {result['accuracy']:<10.4f} {result['epochs']:<8}")


Testing configuration: low_dropout
Reproducible environment set with seed: 9999



Starting training run: comparison_low_dropout_20250813_015447
Using dataset: rated_enhanced_dataset_meta_llama_llama_3.1_8b_instruct_20250811_140130_20250811_151904.jsonl

Loading rated dataset...
Loaded 1756 items with ratings
Dataset after filtering: 1756 items

=== DATASET ANALYSIS ===
Total samples: 1756
Label 0 (Good quality, rating >= 4): 1526 (86.9%)
Label 1 (Poor quality, rating < 4): 230 (13.1%)
Class balance ratio (0:1): 6.63:1

=== RATING DISTRIBUTION ===
Rating 1: 37 (2.1%)
Rating 2: 107 (6.1%)
Rating 3: 86 (4.9%)
Rating 4: 329 (18.7%)
Rating 5: 1197 (68.2%)

Training samples: 1404
Validation samples: 352
DataLoaders created successfully

Initializing model: distilbert-base-uncased
All transformer layers will be fine-tuned
📱 Model moved to device: cuda

=== MODEL PARAMETER ANALYSIS ===
Total parameters: 66,576,257
Trainable parameters: 66,576,257
Frozen parameters: 0
Percentage trainable: 100.0%
LR Annealing enabled: factor=0.5, patience=5

Starting initial validation...
A

Epoch 1/30: 100%|██████████| 88/88 [00:47<00:00,  1.85it/s, loss=0.3028, lr=0.000100]



Epoch 1/30:
  Training   - Loss: 0.4040, Accuracy: 0.8583, F1: 0.0569
  Validation - Loss: 0.3556, Accuracy: 0.8693, F1: 0.0000
  Learning Rate: 0.00010000
No improvement in f1. Patience: 1/500


Epoch 2/30: 100%|██████████| 88/88 [00:47<00:00,  1.84it/s, loss=0.0791, lr=0.000100]



...

Epoch 18/30:
  Training   - Loss: 0.3980, Accuracy: 0.8689, F1: 0.0000
  Validation - Loss: 0.3898, Accuracy: 0.8693, F1: 0.0000
  Learning Rate: 0.00050000
No improvement in f1. Patience: 18/500


Epoch 19/30: 100%|██████████| 88/88 [00:48<00:00,  1.82it/s, loss=0.2801, lr=0.000500]



Epoch 19/30:
  Training   - Loss: 0.4053, Accuracy: 0.8689, F1: 0.0000
  Validation - Loss: 0.4011, Accuracy: 0.8693, F1: 0.0000
  Learning Rate: 0.00050000
No improvement in f1. Patience: 19/500


Epoch 20/30:  86%|████████▋ | 76/88 [00:41<00:06,  1.81it/s, loss=0.7651, lr=0.000500]

: 

## Conclusion: Everything overfits, this method is not appropriate for such a dataset.

### What else could be tried?
- Another embedding model
- Smaller last layer sizes
- Large dataset
- Another framework (papers newer than RouteLLM)

## What if had worked?
- We could explore Performance Gap Recovery
- We could tune Call-Performance Threshold